In [2]:
import re
import os
import json
from random import random
from shutil import copyfile
import time

In [3]:
extract_count = 0

def rename_all_json_in(folderPath):
    newFile=False
    fileList = os.listdir(folderPath)
    f=open(folderPath+'/nextGameNumber.txt','a')
    f.close()
    f=open(folderPath+'/nextGameNumber.txt','r')
    startIndex = f.readlines()
    print(str(len(startIndex)))
    f.close()
    if(len(startIndex)==0):
        startIndex=1
    else:
        startIndex = int(startIndex[0])
    firstIndex=startIndex
    for e in fileList:
         if e.endswith('.json'):
            if re.search('^[0-9]+.json$',e) is None :
                if nbTurnSupp1(folderPath+'/'+e): 
                    os.rename(folderPath+'/'+e,folderPath+'/'+str(startIndex)+'.json')
                    startIndex=startIndex+1
                    newFile=True
                else:
                     os.remove(folderPath+'/'+e)
    f=open(folderPath+'/nextGameNumber.txt','w')
    f.write(str(startIndex))
    f.close()
    return newFile

def nbTurnSupp1(filePath): #True if game last more than 1 turn
    file=open(filePath)
    f=json.loads(file.read())
    if f['plays'][-1]['turn'] >1:
        return True
    else:
        return False 
    

def extract_features(jsonfile,j,dicoPath,oneTurnTwoPlayer):#jsonfile, turn, dictionnary of all monsters, if true : it will aggregate turn of both player
    global extract_count
    print("extracting "+str(extract_count))
    extract_count+=1
    nbrCardMe=0
    nbrCardOpp=0
    monsterDict =json.loads(open(dicoPath, 'r').read())
    if len(jsonfile[j]['my_board'])>0:
        nbrCardMe=len(jsonfile[j]['my_board'])
    if len(jsonfile[j]['opponent_board'])>0:
         nbrCardOpp=len(jsonfile[j]['opponent_board'])
    features=[]        
    #for i in range(0,2*len(monsterDict)):
    max_card_idx = len( [f for f in os.listdir('partiesHS/cards_extract') if re.match(r'^[0-9]', f)])
    for i in range(0,2*max_card_idx):
        features.append('0')       
    myAttack=0
    oppAttack=0
    myTaunt=0
    oppTaunt=0
    myTauntHp=0
    oppTauntHp=0
 
    my_board = jsonfile[j]['my_board']
    opp_board = jsonfile[j]['opponent_board']
    
    my_hand = jsonfile[j]['my_hand']
    opp_hand = jsonfile[j]['opponent_hand']
    
    my_armor = jsonfile[j]['my_armor']
    opp_armor = jsonfile[j]['opponent_armor']
    
    
    card_played = jsonfile[j]['cards_played']
    current_player = jsonfile[j]['current_player']
    
    if current_player == 'me' :
        player = 0
    else:
        player = 1
        
    if oneTurnTwoPlayer :
        card_played_other_player = jsonfile[j-1]['cards_played']
        for i in card_played_other_player :
            if i in monsterDict :
                features[monsterDict[i]['number']*2+((player+1)%2)]=1
        
    for i in card_played :       
        if i in monsterDict :
            features[monsterDict[i]['number']*2+player]=1

    for k in range(0,nbrCardMe):
        monster=jsonfile[j]['my_board'][k]
        monster_name=monster['card_name']
        myAttack=myAttack+monster['card_attack']
               
        if monster_name in monsterDict :
            features[monsterDict[monster_name]['number']*2]='1'
            for power in monsterDict[monster_name]['powers']:
                if(power == 'Taunt'):
                    myTaunt = myTaunt + 1
                    myTauntHp = myTauntHp + monster['card_health']
                    
    for k in range(0,nbrCardOpp):
        monster=jsonfile[j]['opponent_board'][k]
        monster_name=monster['card_name']
        oppAttack=oppAttack+monster['card_attack']
        if monster_name in monsterDict :
           
            features[monsterDict[monster_name]['number']*2+1]='1'
            for power in monsterDict[monster_name]['powers']:
                if(power == 'Taunt'):
                    oppTaunt = oppTaunt + 1
                    oppTauntHp = oppTauntHp + monster['card_health']

    features.insert(0,str(opp_armor))
    features.insert(0,str(my_armor))
    features.insert(0,str(opp_hand))
    features.insert(0,str(my_hand))
    features.insert(0,str(oppTauntHp))
    features.insert(0,str(myTauntHp))
    features.insert(0,str(oppTaunt))
    features.insert(0,str(myTaunt))
    features.insert(0,str(oppAttack))
    features.insert(0,str(myAttack))
    features.insert(0, str(jsonfile[j]['opponent_health']))
    features.insert(0, str(jsonfile[j]['my_health']))
        
    return features
    
def delete_turns_less_than(n,folderPath):#call this in trainset folder to remove games lasting less than n turns
    listTurn = os.listdir(folderPath)
    for element in listTurn:
        file=list(open(folderPath+'/'+element))
        if len(file)<n :
            os.remove(folderPath+'/'+element)
            
def reset_dataset_train(folderPath):
    listTurn = os.listdir(folderPath)
    for f in listTurn :
        dataset_train = open(folderPath+'/'+f,'w')
        dataset_train.close()
        
def create_datasets(folderPath,dicoPath,trainRatio,newFile):
    #game data path , monster dictionnary path , split ratio in order to build train/test set , if set to True : means you have new game file to be added in train/test datasets
    if not os.path.exists(folderPath+'/Datasets_test'):  
         os.mkdir(folderPath+'/Datasets_test')
    if not os.path.exists(folderPath+'/Datasets_train'):  
         os.mkdir(folderPath+'/Datasets_train')
    if not os.path.exists(folderPath+'/Datasets_test_1turn2plays'):  
         os.mkdir(folderPath+'/Datasets_test_1turn2plays')
    if not os.path.exists(folderPath+'/Datasets_train_1turn2plays'):  
         os.mkdir(folderPath+'/Datasets_train_1turn2plays')
            
            
    fileList = os.listdir(folderPath)
    victory=0
    defeat =1
    train =0 
    test =0
    if newFile:
        iNextTestFile = len(os.listdir(folderPath+'/Datasets_test'))+1      
    
        for f in fileList :
            if f.endswith('.json'):
                nb =random()
                file=open(folderPath+'/'+f)
                f=json.loads(file.read())
                result=f['result']
                if result == 'defeat' :
                    result=0
                    defeat = defeat+1
                else :
                    victory=victory+1
                    result=1
                f=f['plays']
                lastTurn = f[-1]['turn']
                f[::-1]
                if(nb<trainRatio):
                    #Training datasets, split by turn
                    train = train+1
                    #1st dataSet
                    for j in range(1,lastTurn):#Ignore first turn : not a real turn
                        dataset_train = open(folderPath+'/Datasets_train/turn'+str(j)+'.txt','a')
                        features = extract_features(f,j,dicoPath,True)
                        dataset_train.write(str(result))
                        for i in features :
                            dataset_train.write(','+str(i))
                            dataset_train.write('\n')
                        dataset_train.close()
            
                    #2nd dataSet
                    j=lastTurn-1
                    k=1
               
                    while j > 1 :  #Aggregate turn of the 2 players, 
                        dataset_train2 = open(folderPath+'/Datasets_train_1turn2plays/turn'+str(k)+'.txt','a')
                        features = extract_features(f,j,dicoPath,True)
                        dataset_train2.write(str(result))
                        for i in features :
                            dataset_train2.write(','+str(i))
                        dataset_train2.write('\n')
                        dataset_train2.close()
                        k=k+1
                        j=j-2
                    file.close()
        
                #Testing datasets, split by game
                else:
                    test = test +1
                    dataset_test = open(folderPath+'/Datasets_test/game'+str(iNextTestFile)+'.txt','w')
                    for j in range(1,lastTurn):  
        
                        features = extract_features(f,j,dicoPath,True)
                        dataset_test.write(str(result))
                        for i in features :
                            dataset_test.write(','+str(i))
                            dataset_test.write('\n')
                    dataset_test.close()
            
                    #2nd dataSets
                    j=lastTurn-1
                    dataset_test2 = open(folderPath+'/Datasets_test_1turn2plays/game'+str(iNextTestFile)+'.txt','w')
                    dataset_test2.close()
                    while j > 1 : 
                        dataset_test2 = open(folderPath+'/Datasets_test_1turn2plays/game'+str(iNextTestFile)+'.txt','a')
                        features = extract_features(f,j,dicoPath,True)  
                        dataset_test2.write(str(result))
                        for i in features :
                            dataset_test2.write(','+str(i))
                        dataset_test2.write('\n')
                        dataset_test2.close()
                        j=j-2
                    file.close()
                    iNextTestFile= iNextTestFile+1
        print('nbGames for train :' + str(train) + ' ratio :'+str(train/(test+train)))
        print('nbGames for test :'+str(test)+ ' ratio :'+str(test/(test+train)))
        if defeat > 1:
            defeat = defeat -1
    else :
        victory =0
        defeat =1
    return victory/(victory+defeat);


def genDataSetTurnMin(folderPath, dicoPath,turnMin,newFile):
    if newFile :
        newFile=rename_all_json_in(folderPath+'/Datasets_GameOf'+str(turnMin)+'Turn')
        create_datasets(folderPath+'/Datasets_GameOf'+str(turnMin)+'Turn',dicoPath,0.8,newFile)
        delete_turns_less_than(2,folderPath+'/Datasets_GameOf'+str(turnMin)+'Turn/Datasets_train_1turn2plays')

def select_all_game_of_N_turn_or_more(folderPath):
    f=open(folderPath+'/lastGameSelected.txt','a')
    f.close()
    gameStart=list(open(folderPath+'/lastGameSelected.txt','r'))
    if gameStart==[]:
        gameStart=[0]

    gameStop=list(open(folderPath+'/nextGameNumber.txt','r'))
    i =0
    j=int(gameStop[0])-1
    print('gamestart [0] ' + str(gameStart[0]) + ' gamestop ' + str(gameStop[0]) )
    # this was typo i think: for j in range (int(gameStart[0])+1,int(gameStop[0])):
    for j in range (1,int(gameStop[0])):
        file=open(folderPath+'/'+str(j)+'.json')
        nbTurn=int((len(json.loads(file.read())['plays'])-2)/2)
        file.close()
        for n in range(1,nbTurn+1):
            if not os.path.exists(folderPath+'/Datasets_GameOf'+str(n)+'Turn'):  
                os.mkdir(folderPath+'/Datasets_GameOf'+str(n)+'Turn')
                i=0
            else:
                i = len(os.listdir(folderPath+'/Datasets_GameOf'+str(n)+'Turn'))# a enregistrer dans un fichier 
            game = open(folderPath+'/Datasets_GameOf'+str(n)+'Turn/game'+str(i+1)+'.json','w')
            copyfile(folderPath+'/'+str(j)+'.json', folderPath+'/Datasets_GameOf'+str(n)+'Turn/game'+str(i+1)+'.json')
            game.close()
            i = i+1
    file=open(folderPath+'/lastGameSelected.txt','w')
    file.write(str(j))
    file.close()

t1 = time.time()

game_folder = 'partiesHS'

newFile=rename_all_json_in(game_folder)
newFile= True
ratio = create_datasets(game_folder,game_folder+'/cards_extract/monsters.json',0.8,newFile)
print(1)
delete_turns_less_than(2,game_folder+'/Datasets_train_1turn2plays')
print(2)
delete_turns_less_than(2,game_folder+'/Datasets_train')
print(3)
select_all_game_of_N_turn_or_more(game_folder)
print(4)
genDataSetTurnMin(game_folder,game_folder+'/cards_extract/monsters.json',9,newFile)
print('ratio victoire :'+str(ratio))

t2 = time.time() - t1
print(t2)

1
extracting 0
extracting 1
extracting 2
extracting 3
extracting 4
extracting 5
extracting 6
extracting 7
extracting 8
extracting 9
extracting 10
extracting 11
extracting 12
extracting 13
extracting 14
extracting 15
extracting 16
extracting 17
extracting 18
extracting 19
extracting 20
extracting 21
extracting 22
extracting 23
extracting 24
extracting 25
extracting 26
extracting 27
extracting 28
extracting 29
extracting 30
extracting 31
extracting 32
extracting 33
extracting 34
extracting 35
extracting 36
extracting 37
extracting 38
extracting 39
extracting 40
extracting 41
extracting 42
extracting 43
extracting 44
extracting 45
extracting 46
extracting 47
extracting 48
extracting 49
extracting 50
extracting 51
extracting 52
extracting 53
extracting 54
extracting 55
extracting 56
extracting 57
extracting 58
extracting 59
extracting 60
extracting 61
extracting 62
extracting 63
extracting 64
extracting 65
extracting 66
extracting 67
extracting 68
extracting 69
extracting 70
extracting 71


extracting 553
extracting 554
extracting 555
extracting 556
extracting 557
extracting 558
extracting 559
extracting 560
extracting 561
extracting 562
extracting 563
extracting 564
extracting 565
extracting 566
extracting 567
extracting 568
extracting 569
extracting 570
extracting 571
extracting 572
extracting 573
extracting 574
extracting 575
extracting 576
extracting 577
extracting 578
extracting 579
extracting 580
extracting 581
extracting 582
extracting 583
extracting 584
extracting 585
extracting 586
extracting 587
extracting 588
extracting 589
extracting 590
extracting 591
extracting 592
extracting 593
extracting 594
extracting 595
extracting 596
extracting 597
extracting 598
extracting 599
extracting 600
extracting 601
extracting 602
extracting 603
extracting 604
extracting 605
extracting 606
extracting 607
extracting 608
extracting 609
extracting 610
extracting 611
extracting 612
extracting 613
extracting 614
extracting 615
extracting 616
extracting 617
extracting 618
extracting

extracting 1097
extracting 1098
extracting 1099
extracting 1100
extracting 1101
extracting 1102
extracting 1103
extracting 1104
extracting 1105
extracting 1106
extracting 1107
extracting 1108
extracting 1109
extracting 1110
extracting 1111
extracting 1112
extracting 1113
extracting 1114
extracting 1115
extracting 1116
extracting 1117
extracting 1118
extracting 1119
extracting 1120
extracting 1121
extracting 1122
extracting 1123
extracting 1124
extracting 1125
extracting 1126
extracting 1127
extracting 1128
extracting 1129
extracting 1130
extracting 1131
extracting 1132
extracting 1133
extracting 1134
extracting 1135
extracting 1136
extracting 1137
extracting 1138
extracting 1139
extracting 1140
extracting 1141
extracting 1142
extracting 1143
extracting 1144
extracting 1145
extracting 1146
extracting 1147
extracting 1148
extracting 1149
extracting 1150
extracting 1151
extracting 1152
extracting 1153
extracting 1154
extracting 1155
extracting 1156
extracting 1157
extracting 1158
extracti

extracting 1620
extracting 1621
extracting 1622
extracting 1623
extracting 1624
extracting 1625
extracting 1626
extracting 1627
extracting 1628
extracting 1629
extracting 1630
extracting 1631
extracting 1632
extracting 1633
extracting 1634
extracting 1635
extracting 1636
extracting 1637
extracting 1638
extracting 1639
extracting 1640
extracting 1641
extracting 1642
extracting 1643
extracting 1644
extracting 1645
extracting 1646
extracting 1647
extracting 1648
extracting 1649
extracting 1650
extracting 1651
extracting 1652
extracting 1653
extracting 1654
extracting 1655
extracting 1656
extracting 1657
extracting 1658
extracting 1659
extracting 1660
extracting 1661
extracting 1662
extracting 1663
extracting 1664
extracting 1665
extracting 1666
extracting 1667
extracting 1668
extracting 1669
extracting 1670
extracting 1671
extracting 1672
extracting 1673
extracting 1674
extracting 1675
extracting 1676
extracting 1677
extracting 1678
extracting 1679
extracting 1680
extracting 1681
extracti

extracting 2138
extracting 2139
extracting 2140
extracting 2141
extracting 2142
extracting 2143
extracting 2144
extracting 2145
extracting 2146
extracting 2147
extracting 2148
extracting 2149
extracting 2150
extracting 2151
extracting 2152
extracting 2153
extracting 2154
extracting 2155
extracting 2156
extracting 2157
extracting 2158
extracting 2159
extracting 2160
extracting 2161
extracting 2162
extracting 2163
extracting 2164
extracting 2165
extracting 2166
extracting 2167
extracting 2168
extracting 2169
extracting 2170
extracting 2171
extracting 2172
extracting 2173
extracting 2174
extracting 2175
extracting 2176
extracting 2177
extracting 2178
extracting 2179
extracting 2180
extracting 2181
extracting 2182
extracting 2183
extracting 2184
extracting 2185
extracting 2186
extracting 2187
extracting 2188
extracting 2189
extracting 2190
extracting 2191
extracting 2192
extracting 2193
extracting 2194
extracting 2195
extracting 2196
extracting 2197
extracting 2198
extracting 2199
extracti

...

extracting 55143
extracting 55144
extracting 55145
extracting 55146
extracting 55147
extracting 55148
extracting 55149
extracting 55150
extracting 55151
extracting 55152
extracting 55153
extracting 55154
extracting 55155
extracting 55156
extracting 55157
extracting 55158
extracting 55159
extracting 55160
extracting 55161
extracting 55162
extracting 55163
extracting 55164
extracting 55165
extracting 55166
extracting 55167
extracting 55168
extracting 55169
extracting 55170
extracting 55171
extracting 55172
extracting 55173
extracting 55174
extracting 55175
extracting 55176
extracting 55177
extracting 55178
extracting 55179
extracting 55180
extracting 55181
extracting 55182
extracting 55183
extracting 55184
extracting 55185
extracting 55186
extracting 55187
extracting 55188
extracting 55189
extracting 55190
extracting 55191
extracting 55192
extracting 55193
extracting 55194
extracting 55195
extracting 55196
extracting 55197
extracting 55198
extracting 55199
extracting 55200
extracting 552

extracting 55635
extracting 55636
extracting 55637
extracting 55638
extracting 55639
extracting 55640
extracting 55641
extracting 55642
extracting 55643
extracting 55644
extracting 55645
extracting 55646
extracting 55647
extracting 55648
extracting 55649
extracting 55650
extracting 55651
extracting 55652
extracting 55653
extracting 55654
extracting 55655
extracting 55656
extracting 55657
extracting 55658
extracting 55659
extracting 55660
extracting 55661
extracting 55662
extracting 55663
extracting 55664
extracting 55665
extracting 55666
extracting 55667
extracting 55668
extracting 55669
extracting 55670
extracting 55671
extracting 55672
extracting 55673
extracting 55674
extracting 55675
extracting 55676
extracting 55677
extracting 55678
extracting 55679
extracting 55680
extracting 55681
extracting 55682
extracting 55683
extracting 55684
extracting 55685
extracting 55686
extracting 55687
extracting 55688
extracting 55689
extracting 55690
extracting 55691
extracting 55692
extracting 556

extracting 56117
extracting 56118
extracting 56119
extracting 56120
extracting 56121
extracting 56122
extracting 56123
extracting 56124
extracting 56125
extracting 56126
extracting 56127
extracting 56128
extracting 56129
extracting 56130
extracting 56131
extracting 56132
extracting 56133
extracting 56134
extracting 56135
extracting 56136
extracting 56137
extracting 56138
extracting 56139
extracting 56140
extracting 56141
extracting 56142
extracting 56143
extracting 56144
extracting 56145
extracting 56146
extracting 56147
extracting 56148
extracting 56149
extracting 56150
extracting 56151
extracting 56152
extracting 56153
extracting 56154
extracting 56155
extracting 56156
extracting 56157
extracting 56158
extracting 56159
extracting 56160
extracting 56161
extracting 56162
extracting 56163
extracting 56164
extracting 56165
extracting 56166
extracting 56167
extracting 56168
extracting 56169
extracting 56170
extracting 56171
extracting 56172
extracting 56173
extracting 56174
extracting 561

extracting 56604
extracting 56605
extracting 56606
extracting 56607
extracting 56608
extracting 56609
extracting 56610
extracting 56611
extracting 56612
extracting 56613
extracting 56614
extracting 56615
extracting 56616
extracting 56617
extracting 56618
extracting 56619
extracting 56620
extracting 56621
extracting 56622
extracting 56623
extracting 56624
extracting 56625
extracting 56626
extracting 56627
extracting 56628
extracting 56629
extracting 56630
extracting 56631
extracting 56632
extracting 56633
extracting 56634
extracting 56635
extracting 56636
extracting 56637
extracting 56638
extracting 56639
extracting 56640
extracting 56641
extracting 56642
extracting 56643
extracting 56644
extracting 56645
extracting 56646
extracting 56647
extracting 56648
extracting 56649
extracting 56650
extracting 56651
extracting 56652
extracting 56653
extracting 56654
extracting 56655
extracting 56656
extracting 56657
extracting 56658
extracting 56659
extracting 56660
extracting 56661
extracting 566

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'partiesHS/Datasets_GameOf9Turn'

In [10]:
def select_all_game_of_N_turn_or_more(folderPath):
    f=open(folderPath+'/lastGameSelected.txt','a')
    f.close()
    gameStart=list(open(folderPath+'/lastGameSelected.txt','r'))
    if gameStart==[]:
        gameStart=[0]

    gameStop=list(open(folderPath+'/nextGameNumber.txt','r'))
    i =0
    j=int(gameStop[0])-1
    print('gamestart [0] ' + str(gameStart[0]) + ' gamestop ' + str(gameStop[0]) )
    # this was typo i think: for j in range (int(gameStart[0])+1,int(gameStop[0])):
    for j in range (1,int(gameStop[0])):
        file=open(folderPath+'/'+str(j)+'.json')
        nbTurn=int((len(json.loads(file.read())['plays'])-2)/2)
        file.close()
        for n in range(1,nbTurn+1):
            if not os.path.exists(folderPath+'/Datasets_GameOf'+str(n)+'Turn'):  
                os.mkdir(folderPath+'/Datasets_GameOf'+str(n)+'Turn')
                i=0
            else:
                i = len(os.listdir(folderPath+'/Datasets_GameOf'+str(n)+'Turn'))# a enregistrer dans un fichier 
            game = open(folderPath+'/Datasets_GameOf'+str(n)+'Turn/game'+str(i+1)+'.json','w')
            copyfile(folderPath+'/'+str(j)+'.json', folderPath+'/Datasets_GameOf'+str(n)+'Turn/game'+str(i+1)+'.json')
            game.close()
            i = i+1
    file=open(folderPath+'/lastGameSelected.txt','w')
    file.write(str(j))
    file.close()
select_all_game_of_N_turn_or_more(game_folder)
genDataSetTurnMin(game_folder,game_folder+'/cards_extract/monsters.json',9,newFile)
print('ratio victoire :'+str(ratio))

t2 = time.time() - t1
print(t2)

gamestart [0] 2060 gamestop 2061
0
extracting 56804
extracting 56805
extracting 56806
extracting 56807
extracting 56808
extracting 56809
extracting 56810
extracting 56811
extracting 56812
extracting 56813
extracting 56814
extracting 56815
extracting 56816
extracting 56817
extracting 56818
extracting 56819
extracting 56820
extracting 56821
extracting 56822
extracting 56823
extracting 56824
extracting 56825
extracting 56826
extracting 56827
extracting 56828
extracting 56829
extracting 56830
extracting 56831
extracting 56832
extracting 56833
extracting 56834
extracting 56835
extracting 56836
extracting 56837
extracting 56838
extracting 56839
extracting 56840
extracting 56841
extracting 56842
extracting 56843
extracting 56844
extracting 56845
extracting 56846
extracting 56847
extracting 56848
extracting 56849
extracting 56850
extracting 56851
extracting 56852
extracting 56853
extracting 56854
extracting 56855
extracting 56856
extracting 56857
extracting 56858
extracting 56859
extracting 56

extracting 57286
extracting 57287
extracting 57288
extracting 57289
extracting 57290
extracting 57291
extracting 57292
extracting 57293
extracting 57294
extracting 57295
extracting 57296
extracting 57297
extracting 57298
extracting 57299
extracting 57300
extracting 57301
extracting 57302
extracting 57303
extracting 57304
extracting 57305
extracting 57306
extracting 57307
extracting 57308
extracting 57309
extracting 57310
extracting 57311
extracting 57312
extracting 57313
extracting 57314
extracting 57315
extracting 57316
extracting 57317
extracting 57318
extracting 57319
extracting 57320
extracting 57321
extracting 57322
extracting 57323
extracting 57324
extracting 57325
extracting 57326
extracting 57327
extracting 57328
extracting 57329
extracting 57330
extracting 57331
extracting 57332
extracting 57333
extracting 57334
extracting 57335
extracting 57336
extracting 57337
extracting 57338
extracting 57339
extracting 57340
extracting 57341
extracting 57342
extracting 57343
extracting 573

extracting 57768
extracting 57769
extracting 57770
extracting 57771
extracting 57772
extracting 57773
extracting 57774
extracting 57775
extracting 57776
extracting 57777
extracting 57778
extracting 57779
extracting 57780
extracting 57781
extracting 57782
extracting 57783
extracting 57784
extracting 57785
extracting 57786
extracting 57787
extracting 57788
extracting 57789
extracting 57790
extracting 57791
extracting 57792
extracting 57793
extracting 57794
extracting 57795
extracting 57796
extracting 57797
extracting 57798
extracting 57799
extracting 57800
extracting 57801
extracting 57802
extracting 57803
extracting 57804
extracting 57805
extracting 57806
extracting 57807
extracting 57808
extracting 57809
extracting 57810
extracting 57811
extracting 57812
extracting 57813
extracting 57814
extracting 57815
extracting 57816
extracting 57817
extracting 57818
extracting 57819
extracting 57820
extracting 57821
extracting 57822
extracting 57823
extracting 57824
extracting 57825
extracting 578

extracting 58251
extracting 58252
extracting 58253
extracting 58254
extracting 58255
extracting 58256
extracting 58257
extracting 58258
extracting 58259
extracting 58260
extracting 58261
extracting 58262
extracting 58263
extracting 58264
extracting 58265
extracting 58266
extracting 58267
extracting 58268
extracting 58269
extracting 58270
extracting 58271
extracting 58272
extracting 58273
extracting 58274
extracting 58275
extracting 58276
extracting 58277
extracting 58278
extracting 58279
extracting 58280
extracting 58281
extracting 58282
extracting 58283
extracting 58284
extracting 58285
extracting 58286
extracting 58287
extracting 58288
extracting 58289
extracting 58290
extracting 58291
extracting 58292
extracting 58293
extracting 58294
extracting 58295
extracting 58296
extracting 58297
extracting 58298
extracting 58299
extracting 58300
extracting 58301
extracting 58302
extracting 58303
extracting 58304
extracting 58305
extracting 58306
extracting 58307
extracting 58308
extracting 583

extracting 58737
extracting 58738
extracting 58739
extracting 58740
extracting 58741
extracting 58742
extracting 58743
extracting 58744
extracting 58745
extracting 58746
extracting 58747
extracting 58748
extracting 58749
extracting 58750
extracting 58751
extracting 58752
extracting 58753
extracting 58754
extracting 58755
extracting 58756
extracting 58757
extracting 58758
extracting 58759
extracting 58760
extracting 58761
extracting 58762
extracting 58763
extracting 58764
extracting 58765
extracting 58766
extracting 58767
extracting 58768
extracting 58769
extracting 58770
extracting 58771
extracting 58772
extracting 58773
extracting 58774
extracting 58775
extracting 58776
extracting 58777
extracting 58778
extracting 58779
extracting 58780
extracting 58781
extracting 58782
extracting 58783
extracting 58784
extracting 58785
extracting 58786
extracting 58787
extracting 58788
extracting 58789
extracting 58790
extracting 58791
extracting 58792
extracting 58793
extracting 58794
extracting 587

...

extracting 86074
extracting 86075
extracting 86076
extracting 86077
extracting 86078
extracting 86079
extracting 86080
extracting 86081
extracting 86082
extracting 86083
extracting 86084
extracting 86085
extracting 86086
extracting 86087
extracting 86088
extracting 86089
extracting 86090
extracting 86091
extracting 86092
extracting 86093
extracting 86094
extracting 86095
extracting 86096
extracting 86097
extracting 86098
extracting 86099
extracting 86100
extracting 86101
extracting 86102
extracting 86103
extracting 86104
extracting 86105
extracting 86106
extracting 86107
extracting 86108
extracting 86109
extracting 86110
extracting 86111
extracting 86112
extracting 86113
extracting 86114
extracting 86115
extracting 86116
extracting 86117
extracting 86118
extracting 86119
extracting 86120
extracting 86121
extracting 86122
extracting 86123
extracting 86124
extracting 86125
extracting 86126
extracting 86127
extracting 86128
extracting 86129
extracting 86130
extracting 86131
extracting 861

extracting 86564
extracting 86565
extracting 86566
extracting 86567
extracting 86568
extracting 86569
extracting 86570
extracting 86571
extracting 86572
extracting 86573
extracting 86574
extracting 86575
extracting 86576
extracting 86577
extracting 86578
extracting 86579
extracting 86580
extracting 86581
extracting 86582
extracting 86583
extracting 86584
extracting 86585
extracting 86586
extracting 86587
extracting 86588
extracting 86589
extracting 86590
extracting 86591
extracting 86592
extracting 86593
extracting 86594
extracting 86595
extracting 86596
extracting 86597
extracting 86598
extracting 86599
extracting 86600
extracting 86601
extracting 86602
extracting 86603
extracting 86604
extracting 86605
extracting 86606
extracting 86607
extracting 86608
extracting 86609
extracting 86610
extracting 86611
extracting 86612
extracting 86613
extracting 86614
extracting 86615
extracting 86616
extracting 86617
extracting 86618
extracting 86619
extracting 86620
extracting 86621
extracting 866

extracting 87054
extracting 87055
extracting 87056
extracting 87057
extracting 87058
extracting 87059
extracting 87060
extracting 87061
extracting 87062
extracting 87063
extracting 87064
extracting 87065
extracting 87066
extracting 87067
extracting 87068
extracting 87069
extracting 87070
extracting 87071
extracting 87072
extracting 87073
extracting 87074
extracting 87075
extracting 87076
extracting 87077
extracting 87078
extracting 87079
extracting 87080
extracting 87081
extracting 87082
extracting 87083
extracting 87084
extracting 87085
extracting 87086
extracting 87087
extracting 87088
extracting 87089
extracting 87090
extracting 87091
extracting 87092
extracting 87093
extracting 87094
extracting 87095
extracting 87096
extracting 87097
extracting 87098
extracting 87099
extracting 87100
extracting 87101
extracting 87102
extracting 87103
extracting 87104
extracting 87105
extracting 87106
extracting 87107
extracting 87108
extracting 87109
extracting 87110
extracting 87111
extracting 871

extracting 87542
extracting 87543
extracting 87544
extracting 87545
extracting 87546
extracting 87547
extracting 87548
extracting 87549
extracting 87550
extracting 87551
extracting 87552
extracting 87553
extracting 87554
extracting 87555
extracting 87556
extracting 87557
extracting 87558
extracting 87559
extracting 87560
extracting 87561
extracting 87562
extracting 87563
extracting 87564
extracting 87565
extracting 87566
extracting 87567
extracting 87568
extracting 87569
extracting 87570
extracting 87571
extracting 87572
extracting 87573
extracting 87574
extracting 87575
extracting 87576
extracting 87577
extracting 87578
extracting 87579
extracting 87580
extracting 87581
extracting 87582
extracting 87583
extracting 87584
extracting 87585
extracting 87586
extracting 87587
extracting 87588
extracting 87589
extracting 87590
extracting 87591
extracting 87592
extracting 87593
extracting 87594
extracting 87595
extracting 87596
extracting 87597
extracting 87598
extracting 87599
extracting 876

extracting 88034
extracting 88035
extracting 88036
extracting 88037
extracting 88038
extracting 88039
extracting 88040
extracting 88041
extracting 88042
extracting 88043
extracting 88044
extracting 88045
extracting 88046
extracting 88047
extracting 88048
extracting 88049
extracting 88050
extracting 88051
extracting 88052
extracting 88053
extracting 88054
extracting 88055
extracting 88056
extracting 88057
extracting 88058
extracting 88059
extracting 88060
extracting 88061
extracting 88062
extracting 88063
extracting 88064
extracting 88065
extracting 88066
extracting 88067
extracting 88068
extracting 88069
extracting 88070
extracting 88071
extracting 88072
extracting 88073
extracting 88074
extracting 88075
extracting 88076
extracting 88077
extracting 88078
extracting 88079
extracting 88080
extracting 88081
extracting 88082
extracting 88083
extracting 88084
extracting 88085
extracting 88086
extracting 88087
extracting 88088
extracting 88089
extracting 88090
extracting 88091
extracting 880